In [212]:
import json
import glob
import numpy as np
import pandas as pd
import csv
import string
import re
import math
import sklearn as sk
import nltk
import heapq
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfTransformer# This Python 3 environment comes with many helpful analytics libraries installed
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import seaborn as sns

In [213]:
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [214]:
train_files = glob.glob("../input/sample1/SmartLawDataset-main/json/*.json")

In [215]:
mergejson=pd.DataFrame()
for file in train_files:
    with open(file) as f1: 
        data1 = json.load(f1)
    df1=pd.DataFrame([data1])
    mergejson=pd.concat([mergejson,df1])
mg1=mergejson.drop(['header','background','order','footer','annotationProcessingStage','annotationProcessingStageAnnotations','processedText'], axis=1)
mg1.to_csv('file2.csv')
#print(mergejson)

In [216]:
header=['ARGUMENT','TYPE']
filename = 'final_argument_by.csv'
data1=[]

with open(filename, 'w') as file:
    csvwriter = csv.writer(file) # 2. create a csvwriter object
    csvwriter.writerow(header) # 4. write the header
    for j in mg1['arguments']:
        for k in j:           
            for p in k['argumentSentences']:                
                data=[]
                if(p['argumentSentenceType']=='NA'):
                    continue
                data.append(p['text'][:])
                data.append(p['argumentSentenceType'])
                data1.append(data)
    csvwriter.writerows(data1) # 5. write the rest of the data
dataset = pd.read_csv('final_argument_by.csv')
df = pd.DataFrame(dataset)

In [217]:
s =  set(string.punctuation)
s.add('\xad')
nltk_stopwords = set(stopwords.words('english'))
for index, row in df.iterrows():
    row['ARGUMENT']=row['ARGUMENT'].lower()
    for x in row['ARGUMENT']:
        if x in s or re.search(r'-?\d+', x):
            row['ARGUMENT']=row['ARGUMENT'].replace(x,"").strip()
for index, row in df.iterrows():
    for x in row['ARGUMENT'].split():
        if x in nltk_stopwords:
            row['ARGUMENT']=row['ARGUMENT'].replace(x,"").strip()
        
#Now we have proper dataset after removing unwanted punctuations etc
arr=[]
arr1=[]
count=0
c1=0
for index, row in df.iterrows():
    if(row['TYPE']=='PREMISE'):
        count+=1
        if(count>290):
            continue
    arr.append(row['ARGUMENT'])
    arr1.append(row['TYPE'])

df1=pd.DataFrame()
df1["argument"] = arr
df1["by"] = arr1
print(df1)

In [218]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df1['argument'])
columns=count_vect.get_feature_names_out()
fin_x=pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names_out())
print(fin_x)

In [219]:
final_by=[]
for item in df1['by']:
    final_by.append(item)
header=columns
filename = 'final_csv_1.csv'
data1=[]
with open(filename, 'w',encoding="UTF-8") as file:
    csvwriter = csv.writer(file) # 2. create a csvwriter object
    csvwriter.writerow(header) # 4. write the header
    for item in X_counts.toarray():
        data1.append(item)
    csvwriter.writerows(data1) # 5. write the rest of the data
final_bow=pd.read_csv(filename)
final_bow['TYpe']=df1['by']
final_bow

In [220]:
final_bow = final_bow.dropna(axis=0)

In [221]:
y=final_bow.TYpe
features=header
x=final_bow[features]

In [222]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)
# check the shape of X_train and X_test
X_train.shape, X_test.shape

In [223]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

In [224]:
y_pred = gnb.predict(X_test)
y_pred_train = gnb.predict(X_train)

In [225]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [226]:
print(y_test.value_counts())
null_accuracy = (76/(69+76))
print('Null accuracy score: {0:0.4f}'. format(null_accuracy))

In [227]:
#SVC model
lsvc = LinearSVC(verbose=0)
print(lsvc)

In [228]:
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [229]:
lsvc.fit(X_train, y_train)
score = lsvc.score(X_train, y_train)
print("Score: ", score)

In [230]:
knn = KNeighborsClassifier(n_neighbors= 3) # k=3
knn.fit(X_train,y_train)
prediction = knn.predict(X_test)

In [231]:
print("{} nın score: {}".format(3,knn.score(X_test,y_test)))

In [232]:
def learn_curve(X,y,c):
    le = preprocessing.LabelEncoder() # Label encoding the target
    sc = preprocessing.StandardScaler() # Scaling the input features
    y = le.fit_transform(y)#Label Encoding the target
    log_reg = LogisticRegression(max_iter=200,random_state=11,C=c)
    lr = Pipeline(steps=(['scaler',sc],
                        ['classifier',log_reg]))
    
    
    cv = StratifiedKFold(n_splits=5,random_state=11,shuffle=True) # Creating a StratifiedKFold object with 5 folds
    cv_scores = cross_val_score(lr,X,y,scoring="accuracy",cv=cv) # Storing the CV scores (accuracy) of each fold
    lr.fit(X,y) # Fitting the model
    train_score = lr.score(X,y) # Scoring the model on train set
    #Building the learning curve
    train_size,train_scores,test_scores = learning_curve(estimator=lr,X=X,y=y,cv=cv,scoring="accuracy",random_state=11)
    train_scores = 1-np.mean(train_scores,axis=1)#converting the accuracy score to misclassification rate
    test_scores = 1-np.mean(test_scores,axis=1)#converting the accuracy score to misclassification rate
    lc = pd.DataFrame({"Training_size":train_size,"Training_loss":train_scores,"Validation_loss":test_scores}).melt(id_vars="Training_size")
    return {"cv_scores":cv_scores,
           "train_score":train_score,
           "learning_curve":lc}

In [233]:
lc = learn_curve(x,y,1)
print(f'Cross Validation Accuracies:\n{"-"*25}\n{list(lc["cv_scores"])}\n\n\
Mean Cross Validation Accuracy:\n{"-"*25}\n{np.mean(lc["cv_scores"])}\n\n\
Standard Deviation of Cross Validation Accuracy:\n{"-"*25}\n{np.std(lc["cv_scores"])}\n\n\
Training Accuracy:\n{"-"*15}\n{lc["train_score"]}\n\n')
sns.lineplot(data=lc["learning_curve"],x="Training_size",y="value",hue="variable")
plt.title("Learning Curve of Model")
plt.ylabel("Misclassification Rate/Loss");